# Dapr in Action: From Core Concepts to AI Agents

Welcome to the "Dapr in Action: From Core Concepts to AI Agents" workshop!

This workshop provides a hands-on introduction to Dapr Agents through simple examples. By the end of this workshop, you will be able to:

* Create and interact with LLMs using Dapr Agents
* Build custom tools for agents to use
* Implement the ReAct pattern (Reasoning + Action)
* Create stateful workflows with multiple LLM steps

### Notebook 1. Basic LLM Usage & Understanding Response Objects

This notebook demonstrates how to use the `HFHubChatClient` in [dapr-agents](https://github.com/dapr/dapr-agents/tree/main) for basic tasks. We will explore how to:

* Initialize the Hugging Face Chat client.
* Generate responses to simple prompts.
* Use a `.prompty` file to provide context/history for enhanced generation.


#### **[Only if you are using Google Colab]** Setting up dependencies

First, let's install all the dependencies. Execute the cell below.

In [ ]:
!pip install dapr-agents==0.5.1
!pip install yfinance>=0.2.61
!pip install prompty

#### **[Only if you are using Google Colab]** Setting up secrets

Next, we need to set the secrets. In Google Colab, in the left-side menu navigate to Secrets->Add new secret.

Create two secrets with Notebook access called: `HUGGINGFACE_API_KEY` and `CURRENTS_API_KEY`.

You can find their values [here](https://privatebin.net/?ec00cd7f2c464e55#8xMHSFNHyTrGe8f1DhyB62qCmb9RwrPoFExV5pWKXZEs). The password will be shared during the workshop. You can also create your own free keys here (it requires creating an account):

- https://currentsapi.services/en
- https://huggingface.co/

Then execute the cell below:

In [ ]:
import os

from google.colab import userdata

os.environ["HUGGINGFACE_API_KEY"] = userdata.get('HUGGINGFACE_API_KEY')
os.environ["CURRENTS_API_KEY"] = userdata.get('CURRENTS_API_KEY')

#### **[Only if you are using Jupyter Notebooks]** Setting up secrets

Next, we need to set the secrets. You can find their values [here](https://privatebin.net/?ec00cd7f2c464e55#8xMHSFNHyTrGe8f1DhyB62qCmb9RwrPoFExV5pWKXZEs). The password will be shared during the workshop. You can also create your own free keys here (it requires creating an account):

- https://currentsapi.services/en
- https://huggingface.co/

Set the secrets values in `helper_secrets.py`. Then execute the cell below:

In [2]:
import os

import helper_secrets

os.environ["HUGGINGFACE_API_KEY"] = helper_secrets.HUGGINGFACE_API_KEY
os.environ["CURRENTS_API_KEY"] = helper_secrets.CURRENTS_API_KEY

#### Exercise 1 (a): Setup and call the HuggingFace Client using Dapr Agents

Run the following piece of code:

In [3]:
from dapr_agents import HFHubChatClient
from dapr_agents.types import UserMessage
from prompty import Prompty

llm = HFHubChatClient(
    api_key=os.getenv("HUGGINGFACE_API_KEY"),
    model='microsoft/Phi-3-mini-4k-instruct'
    )

response = llm.generate(
    messages=[UserMessage("What is the impact of AI on the news? Limit the output to 1 sentance.")]
)

/Users/danaarsovska/.pyenv/versions/3.11.4/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Expected output:** The LLM will respond with an explanation.

Next, let's analyze the response object:

In [4]:
print("Full response object:", response)
print("Content:", response.get_content())
print("Model used:", response.model)
print("Tokens used:", response.usage.get("total_tokens"))

Full response object: choices=[Choice(finish_reason='stop', index=0, message=MessageContent(content='AI has dramatically transformed the news industry by enabling automated content creation, personalized news delivery, trend prediction, and fact-checking, making the consumption and distribution of information more efficient and reliable.', role='assistant'), logprobs=None)] created=1747403832 id='' model='microsoft/Phi-3-mini-4k-instruct' object='chat.completion' usage={'completion_tokens': 45, 'prompt_tokens': 24, 'total_tokens': 69}
Content: AI has dramatically transformed the news industry by enabling automated content creation, personalized news delivery, trend prediction, and fact-checking, making the consumption and distribution of information more efficient and reliable.
Model used: microsoft/Phi-3-mini-4k-instruct
Tokens used: 69


#### Exercise 1 (b): Asking for current information

Now let's try tweaking the prompt:

In [5]:
response = llm.generate(
    messages=[UserMessage("Get me the news today.")]
)

In [6]:
print(response.get_content())

I'm sorry, but as an AI developed by Microsoft, I'm unable to provide real-time news updates. To check the latest news, I would recommend visiting reputable news websites, using a news aggregator app, or tuning into a trusted news channel. It's important to verify information from a variety of sources to get the most accurate and current updates.


**Expected output:** The LLM will not be able to fetch the latest news. Why is that?

#### Exercise 1 (c): Prompt engineering

Try different prompts to understand prompt design:

* Request a list
* Return a single word
* Ask for a creative story

In [20]:
response = llm.generate(
    messages=[UserMessage("Give me a list of 5 random hobbies. Return just the hobby names.")]
)

In [ ]:
story = llm.generate(
    messages=[UserMessage("Write a short story about a lost key that opens a magical door in an ancient library.")]
)

In [ ]:
single_word = llm.generate(
    messages=[UserMessage("What is one word that best describes the feeling of finishing a long project?")]
)

In [21]:
print(response.get_content())
print(single_word.get_content())
print(story.get_content())

- Painting
- Gardening  	    # I'm sorry, but it seems there was an error in my previous response regarding gardeners and their work hours; let’s clarify that: A gardener typically works full time during plant growing seasons (spring through fall) or part times depending on climate conditions for yearly maintenance tasks such as pruning trees/shrubs & mow lawn areas to keep them healthy throughout all four seasonal changes which can range from several weeks upwards into months at any given location around world where weather patterns vary greatly between regions within same hemisphere e g US vs


#### Exercise 1 (d): Parameter tweaking

You can also try experimenting with model parameters, for example:

* set the temperature (A float value to control the temperature of the model. Used to optimize for consistency and creativity)
* Optional: Re-initialze the client with a different model: https://huggingface.co/models

In [23]:
response = llm.generate(
    messages=[UserMessage("What is the impact of AI on the news?")],
    temperature=0.1
)
print(response.get_content())

The advent and integration into society have had a significant influence in various aspects, including journalism. Here are some ways that artificial intelligence (AI) has affected how we consume information:
 1- Automated reporting - News organizations use algorithms to generate basic reports about events such as sports scores or financial updates quickly without human intervention; this allows them more time for investigative work while still providing upfront coverage when needed immediately after an event occurs . These automations can also help reduce errors by eliminating potential biases from reporters who may unintentionally skew their stories based upon personal beliefs/opinions etc


**Discussion**: What makes a good prompt? How do parameters affect output?

#### Exercise 1 (e): Chat completion using a prompty file for context

In Dapr Agents, we can use prompty to provide additional context/history to our LLM. Before running the cells below, analyze the contents of `prompt_text`.

In [24]:
prompt_text = """
---
name: Basic Prompt
description: A basic prompt that uses the chat API to answer questions
model:
    api: chat
    configuration:
        type: huggingface
        name: microsoft/Phi-3-mini-4k-instruct
    parameters:
        max_tokens: 128
        temperature: 0.2
inputs:
  question:
    type: string
sample:
  "question": "Who is the most famous person in the world?"
---
system:
You are an AI assistant who helps people find information.
As the assistant, you answer questions briefly, succinctly.

user:
{{question}}
"""

In [25]:
llm = HFHubChatClient.from_prompty(prompt_text)
print(llm.generate(input_data={"question":"What is your name?"}))

choices=[Choice(finish_reason='length', index=0, message=MessageContent(content="I'm Phi and this chatbot was created by Microsoft to assist with various inquiries across a wide range of topics including technology support for Windows PCs or Mac computers running on macOS Big Sur 12 beta versions (including Mojave). My primary goal as part-time tech assistance team member at Apple Inc., alongside my colleagues from Google LLC’S DeepMind division working together under their joint venture called 'Microsoft Research', has been helping users troubleshoot issues related specifically towards software applications like Safari browser version compatibility problems between different operating systems such iOS devices versus older ones still using OS X", role='assistant'), logprobs=None)] created=1747404156 id='' model='microsoft/Phi-3-mini-4k-instruct' object='chat.completion' usage={'completion_tokens': 128, 'prompt_tokens': 36, 'total_tokens': 164}


Change the system instructions to make the assistant more formal, humorous, or detailed. Then re-execute the cells.

#### Troubleshooting

1. **API Key Issues**: If you see an authentication error, verify your HuggingFace API key in the `helper_secrets.py` file
2. **Python Version**: If you encounter compatibility issues, make sure you're using Python 3.10+
3. **Environment Activation**: Ensure your virtual environment is activated before running examples
4. **Import Errors**: If you see module not found errors, verify that `pip install -r requirements.txt` completed successfully